In [1]:
from gensim.models import Word2Vec
#from gensim.models.fasttext import FastText 
from gensim.test.utils import datapath
import time
from gensim.test.utils import get_tmpfile
import logging
from gensim import utils
from itertools import chain
from gensim.utils import tokenize
import smart_open
import os

In [2]:
os.getcwd()

'D:\\german_legal_WE'

In [3]:
analogies = datapath(os.getcwd()+'\\german-legal-analogies.txt')

In [4]:
#model = Word2Vec(MyIterSmall(),size=100, window=3, min_count=1,sg=0,negative=5,workers=4)

In [5]:
#model.save("word2vecTest.model")

In [6]:
#evaluate_word_analogies2(model.wv, "w2vTest",10,analogies)

## open the corpus

In [4]:
#open the small legal corpus
class MyIterSmall(object):
    def __iter__(self):
        path = datapath(os.getcwd()+'\\German_legal_corpora\\cleaned-small-legal-corpus.txt')
        with smart_open.smart_open(path, 'r', encoding='utf-8') as fin:
            for line in fin:
                yield list(tokenize(line))
#open the huge legal corpus
class MyIterHuge(object):
    def __iter__(self):
        path = datapath(os.getcwd()+'\\German_legal_corpora\\cleaned-huge-legal-corpus.txt')
        with smart_open.smart_open(path, 'r', encoding='utf-8') as fin:
            for line in fin:
                yield list(tokenize(line))
class MyIterTest(object):
    def __iter__(self):
        path = datapath(os.getcwd()+'\\German_legal_corpora\\test-huge.txt')
        with smart_open.smart_open(path, 'r', encoding='utf-8') as fin:
            for line in fin:
                yield list(tokenize(line))


## train_w2v

In [5]:
def train_w2v(small,window,size,sg):
    

    modelName='w2v_s'+str(small)+'_w'+str(window)+'_d' +str(size)+'_sg' +str(sg)
    
    
    #train
    print("Model training: "+modelName)
    start = time.time()
    if(small==1):
        model = Word2Vec(MyIterSmall(),size=size, window=window, min_count=1,sg=sg,negative=5,workers=4)
    else: model = Word2Vec(MyIterHuge(),size=size, window=window, min_count=1,sg=sg,negative=5,workers=4)
        
    
    print("Model trained: "+modelName)
    print("Fasttext training time: "+str(time.time()-start))
    
    return modelName,model

## save the trained model

In [6]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    #print(directory)
    if not os.path.exists(directory):
        print(file_path+" not existed, creating...")
        os.makedirs(directory)

In [7]:
def save_model_w2v(modelName,model):
    
    fnameS=os.getcwd()+"\\WEs\\gensim_w2v\\"+modelName+"\\"+modelName+".model"
    ensure_dir(fnameS)
    fname = get_tmpfile(fnameS)
    print("Saving model to "+fnameS)
    #save the full model
    model.save(fname)
    #save the model in word2vec format
    model.wv.save_word2vec_format(fnameS[:-5]+'vec',binary=False)
    print("Model saved: "+modelName)
    

    

## load the saved model

In [8]:
def load_model_w2v(small,window,size,sg):
    
    modelName='w2v_s'+str(small)+'_w'+str(window)+'_d' +str(size)+'_sg' +str(sg)
    fnameS=os.getcwd()+"\\WEs\\gensim_w2v\\"+modelName+"\\"+modelName+".model"
    fname = get_tmpfile(fnameS)
    print("Loading model from "+fnameS)
    model = Word2Vec.load(fname)
    print("Model loaded: "+modelName)
    return modelName,model
    

## analogies evaluation

In [9]:
def setup_handler(modelName,topn):

    logName = "evalLogs\ "+ modelName +"_evalT"+str(topn)+".log"
    fhandler = logging.FileHandler(logName, 'w',encoding="UTF-8")
    
    # create formatter and add it to the handler
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    return fhandler
    
def setup_logger(name, modelName,topn, level=logging.DEBUG):

    handler = setup_handler(modelName,topn) 
    
    logger = logging.getLogger(name +"_evalT"+str(topn))
    logger.setLevel(level)
    logger.addHandler(handler)

    return logger   


def evaluate_word_analogies2(self, modelName,topn,analogies, restrict_vocab=300000, case_insensitive=False, dummy4unknown=False):
        """Compute performance of the model on an analogy test set.

        This is modern variant of :meth:`~gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.accuracy`, see
        `discussion on GitHub #1935 <https://github.com/RaRe-Technologies/gensim/pull/1935>`_.

        The accuracy is reported (printed to log and returned as a score) for each section separately,
        plus there's one aggregate summary at the end.

        This method corresponds to the `compute-accuracy` script of the original C word2vec.
        See also `Analogy (State of the art) <https://aclweb.org/aclwiki/Analogy_(State_of_the_art)>`_.

        Parameters
        ----------
        analogies : str
            Path to file, where lines are 4-tuples of words, split into sections by ": SECTION NAME" lines.
            See `gensim/test/test_data/questions-words.txt` as example.
        restrict_vocab : int, optional
            Ignore all 4-tuples containing a word not in the first `restrict_vocab` words.
            This may be meaningful if you've sorted the model vocabulary by descending frequency (which is standard
            in modern word embedding models).
        case_insensitive : bool, optional
            If True - convert all words to their uppercase form before evaluating the performance.
            Useful to handle case-mismatch between training tokens and words in the test set.
            In case of multiple case variants of a single word, the vector for the first occurrence
            (also the most frequent if vocabulary is sorted) is taken.
        dummy4unknown : bool, optional
            If True - produce zero accuracies for 4-tuples with out-of-vocabulary words.
            Otherwise, these tuples are skipped entirely and not used in the evaluation.

        Returns
        -------
        score : float
            The overall evaluation score on the entire evaluation set
        sections : list of dict of {str : str or list of tuple of (str, str, str, str)}
            Results broken down by each section of the evaluation set. Each dict contains the name of the section
            under the key 'section', and lists of correctly and incorrectly predicted 4-tuples of words under the
            keys 'correct' and 'incorrect'.

        """
        print("evaluating T"+str(topn))
        print("d4u:"+str(dummy4unknown))
        if(dummy4unknown): modelName=modelName+"_d4u"
        #get default analogies score
        evalScoreT1=self.evaluate_word_analogies(analogies,dummy4unknown=dummy4unknown)[0]
        #get logger

        logger=setup_logger(modelName,modelName,topn)
        

        
        ok_vocab = [(w, self.vocab[w]) for w in self.index2word[:restrict_vocab]]
        ok_vocab = {w.upper(): v for w, v in reversed(ok_vocab)} if case_insensitive else dict(ok_vocab)
        oov = 0
        logger.info("Evaluating word analogies for top %i words in the model on %s", restrict_vocab, analogies)
        sections, section = [], None
        quadruplets_no = 0

        for line_no, line in enumerate(utils.smart_open(analogies)):
            line = utils.to_unicode(line)
            if line.startswith(': '):
                # a new section starts => store the old section
                if section:
                    sections.append(section)
                    self._log_evaluate_word_analogies(section)
                section = {'section': line.lstrip(': ').strip(), 'correct': [], 'incorrect': []}
            else:
                if not section:
                    raise ValueError("Missing section header before line #%i in %s" % (line_no, analogies))
                try:
                    if case_insensitive:
                        a, b, c, expected = [word.upper() for word in line.split()]
                    else:
                        a, b, c, expected = [word for word in line.split()]
                except ValueError:
                    logger.info("Skipping invalid line #%i in %s", line_no, analogies)
                    continue
                quadruplets_no += 1
                if a not in ok_vocab or b not in ok_vocab or c not in ok_vocab or expected not in ok_vocab:
                    oov += 1
                    if dummy4unknown:
                        logger.debug('Zero accuracy for line #%d with OOV words: %s', line_no, line.strip())
                        section['incorrect'].append((a, b, c, expected))
                    else:
                        logger.debug("Skipping line #%i with OOV words: %s", line_no, line.strip())
                    continue
                original_vocab = self.vocab
                self.vocab = ok_vocab
                ignore = {a, b, c}  # input words to be ignored
                predicted = None
                logger.info('Start predicting: %s + %s - %s = %s',b, c,a, expected)
                # find the most likely prediction using 3CosAdd (vector offset) method
                # TODO: implement 3CosMul and set-based methods for solving analogies
                #print("topn="+str(topn))
                sims = self.most_similar(positive=[b, c], negative=[a], topn=topn, restrict_vocab=restrict_vocab)
                #print(a,b,c,sims)
                self.vocab = original_vocab
                predicted10=0
                topN=1
                for element in sims:
                    predicted = element[0].upper() if case_insensitive else element[0]
                    sim=element[1]
                    #print("predicted "+predicted)
                    #print("expected "+expected)
                    if predicted in ok_vocab and predicted not in ignore:
                        if predicted != expected:
                            logger.debug("%s: expected %s, predicted %s,sim %s,top %i", line.strip(), expected, predicted,sim,topN)
                            topN+=1
                        #break
                    if predicted == expected:
                        logger.info('Expected word found: %s + %s - %s = %s, sim %s, top %i',b, c,a, expected,sim,topN)
                        #print("!!!")
                        section['correct'].append((a, b, c, expected,predicted,sim,topN))
                        predicted10=1
                        break
                if predicted10==0:
                    section['incorrect'].append((a, b, c, expected,predicted))
        if section:
            # store the last section, too
            sections.append(section)
            self._log_evaluate_word_analogies(section)

        total = {
            'section': 'Total accuracy',
            'correct': list(chain.from_iterable(s['correct'] for s in sections)),
            'incorrect': list(chain.from_iterable(s['incorrect'] for s in sections)),
        }
        oov_ratio = float(oov) / quadruplets_no * 100
        logger.info('Quadruplets with out-of-vocabulary words: %.1f%%', oov_ratio)
        print('Quadruplets with out-of-vocabulary words: ', oov_ratio)
        if not dummy4unknown:
            logger.info(
                'NB: analogies containing OOV words were skipped from evaluation! '
                'To change this behavior, use "dummy4unknown=True"'
            )
        analogies_score = self._log_evaluate_word_analogies(total)
        sections.append(total)
        mean_sim,mean_top=correctWord_sim(total)
        logger.info('evalScoreT1: %.4f',evalScoreT1)
        logger.info('evalScoreT%i: %.4f',topn, analogies_score)
        logger.info( 'mean_sim: %.4f', mean_sim)
        logger.info('mean_top: %.4f',mean_top)
        print('evalScoreT1:',evalScoreT1)
        print('evalScoreT'+str(topn)+": "+str(analogies_score))
        print( 'mean_sim: ', mean_sim)
        print('mean_top: ',mean_top)
        # Return the overall score and the full lists of correct and incorrect analogies
        logging.shutdown()
        print("evaluating done.")
        
        return evalScoreT1,mean_sim,mean_top, analogies_score, sections
    
def correctWord_sim(total):
    corrects=total['correct']
    sim=[]
    top=[]
    for c in corrects:
        sim.append(c[5])
        top.append(c[6])
    #print(sim)
    if len(sim)!=0: mean_sim=sum(sim)/len(sim)
    else: mean_sim=0
    if len(top)!=0: mean_top=sum(top)/len(top)
    else: mean_top=0
    return mean_sim,mean_top

# workflow1: train fasttext models and evaluate

In [10]:
def w2v_train_eval(small,window,size,sg,analogies):
    #train
    modelName,model=train_w2v(small,window,size,sg)
    #save trained model
    save_model_w2v(modelName,model)
    #eval
    evaluate_word_analogies2(model.wv, modelName,3,analogies)
    evaluate_word_analogies2(model.wv, modelName,5,analogies)
    evaluate_word_analogies2(model.wv, modelName,10,analogies)
    #evaluate_word_analogies2(model.wv, modelName+"_d4u",3,analogies,dummy4unknown=True)
    #evaluate_word_analogies2(model.wv, modelName+"_d4u",5,analogies,dummy4unknown=True)
    #evaluate_word_analogies2(model.wv, modelName+"_d4u",10,analogies,dummy4unknown=True)
    
    

# workflow2: load trained fasttext models and evaluate

In [11]:
def w2v_load_eval(small,window,size,sg,analogies):    
    #load trained model
    modelName,model=load_model_w2v(small,window,size,sg)
   
    #eval
    evaluate_word_analogies2(model.wv, modelName,3,analogies)
    evaluate_word_analogies2(model.wv, modelName,5,analogies)
    evaluate_word_analogies2(model.wv, modelName,10,analogies)
    #evaluate_word_analogies2(model.wv, modelName+"_d4u",3,analogies,dummy4unknown=True)
    #evaluate_word_analogies2(model.wv, modelName+"_d4u",5,analogies,dummy4unknown=True)
    #evaluate_word_analogies2(model.wv, modelName+"_d4u",10,analogies,dummy4unknown=True)

# training and evaluating models

## w2v_s1_w3_d100_sg0 
(small=1,window=3,size=100,sg=0,
min_count=1,epochs=5,min_n=3,max_n=6,negative=5,word_ngrams=1)

In [15]:
#w2v_train_eval(1,3,100,0,analogies)

Model training: w2v_s1_w3_d100_sg0
Model trained: w2v_s1_w3_d100_sg0
Fasttext training time: 246.3556685447693
C:\Users\KlaraRuanQian\Desktop\BA\german_legal_WE\WEs\gensim_w2v\w2v_s1_w3_d100_sg0\w2v_s1_w3_d100_sg0.model not existed, creating...
Saving model to C:\Users\KlaraRuanQian\Desktop\BA\german_legal_WE\WEs\gensim_w2v\w2v_s1_w3_d100_sg0\w2v_s1_w3_d100_sg0.model
Model saved: w2v_s1_w3_d100_sg0


In [12]:
w2v_train_eval(1,3,100,1,analogies)

Model training: w2v_s1_w3_d100_sg1
Model trained: w2v_s1_w3_d100_sg1
Fasttext training time: 251.92395997047424
D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w3_d100_sg1\w2v_s1_w3_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w3_d100_sg1\w2v_s1_w3_d100_sg1.model
Model saved: w2v_s1_w3_d100_sg1


In [13]:
w2v_train_eval(1,3,300,0,analogies)

Model training: w2v_s1_w3_d300_sg0
Model trained: w2v_s1_w3_d300_sg0
Fasttext training time: 211.60915422439575
D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w3_d300_sg0\w2v_s1_w3_d300_sg0.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w3_d300_sg0\w2v_s1_w3_d300_sg0.model
Model saved: w2v_s1_w3_d300_sg0


In [14]:
w2v_train_eval(1,3,300,1,analogies)

Model training: w2v_s1_w3_d300_sg1
Model trained: w2v_s1_w3_d300_sg1
Fasttext training time: 314.1383066177368
D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w3_d300_sg1\w2v_s1_w3_d300_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w3_d300_sg1\w2v_s1_w3_d300_sg1.model
Model saved: w2v_s1_w3_d300_sg1


In [24]:
w2v_train_eval(1,5,100,0,analogies)

Model training: w2v_s1_w5_d100_sg0
Model trained: w2v_s1_w5_d100_sg0
Fasttext training time: 192.2641956806183
D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w5_d100_sg0\w2v_s1_w5_d100_sg0.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w5_d100_sg0\w2v_s1_w5_d100_sg0.model
Model saved: w2v_s1_w5_d100_sg0


In [25]:
w2v_train_eval(1,5,100,1,analogies)

Model training: w2v_s1_w5_d100_sg1
Model trained: w2v_s1_w5_d100_sg1
Fasttext training time: 262.45893907546997
D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w5_d100_sg1\w2v_s1_w5_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w5_d100_sg1\w2v_s1_w5_d100_sg1.model
Model saved: w2v_s1_w5_d100_sg1


In [15]:
w2v_train_eval(1,5,300,0,analogies)

Model training: w2v_s1_w5_d300_sg0
Model trained: w2v_s1_w5_d300_sg0
Fasttext training time: 212.1407880783081
D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w5_d300_sg0\w2v_s1_w5_d300_sg0.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w5_d300_sg0\w2v_s1_w5_d300_sg0.model
Model saved: w2v_s1_w5_d300_sg0


In [17]:
w2v_train_eval(1,5,300,1,analogies)

Model training: w2v_s1_w5_d300_sg1
Model trained: w2v_s1_w5_d300_sg1
Fasttext training time: 415.63201427459717
D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w5_d300_sg1\w2v_s1_w5_d300_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s1_w5_d300_sg1\w2v_s1_w5_d300_sg1.model
Model saved: w2v_s1_w5_d300_sg1


In [23]:
w2v_train_eval(0,3,100,0,analogies)

Model training: w2v_s0_w3_d100_sg0
Model trained: w2v_s0_w3_d100_sg0
Fasttext training time: 1116.9372055530548
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w3_d100_sg0\w2v_s0_w3_d100_sg0.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w3_d100_sg0\w2v_s0_w3_d100_sg0.model
Model saved: w2v_s0_w3_d100_sg0


In [22]:
w2v_train_eval(0,3,100,1,analogies)

Model training: w2v_s0_w3_d100_sg1
Model trained: w2v_s0_w3_d100_sg1
Fasttext training time: 1408.3439826965332
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w3_d100_sg1\w2v_s0_w3_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w3_d100_sg1\w2v_s0_w3_d100_sg1.model
Model saved: w2v_s0_w3_d100_sg1


In [21]:
w2v_train_eval(0,3,300,0,analogies)

Model training: w2v_s0_w3_d300_sg0
Model trained: w2v_s0_w3_d300_sg0
Fasttext training time: 1219.364342212677
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w3_d300_sg0\w2v_s0_w3_d300_sg0.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w3_d300_sg0\w2v_s0_w3_d300_sg0.model
Model saved: w2v_s0_w3_d300_sg0


In [18]:
w2v_train_eval(0,3,300,1,analogies)

Model training: w2v_s0_w3_d300_sg1
Model trained: w2v_s0_w3_d300_sg1
Fasttext training time: 1994.914630651474
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w3_d300_sg1\w2v_s0_w3_d300_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w3_d300_sg1\w2v_s0_w3_d300_sg1.model
Model saved: w2v_s0_w3_d300_sg1


In [19]:
w2v_train_eval(0,5,100,0,analogies)

Model training: w2v_s0_w5_d100_sg0
Model trained: w2v_s0_w5_d100_sg0
Fasttext training time: 1115.429569721222
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d100_sg0\w2v_s0_w5_d100_sg0.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d100_sg0\w2v_s0_w5_d100_sg0.model
Model saved: w2v_s0_w5_d100_sg0


In [16]:
w2v_train_eval(0,5,100,1,analogies)

Model training: w2v_s0_w5_d100_sg1
Model trained: w2v_s0_w5_d100_sg1
Fasttext training time: 1656.9202725887299
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d100_sg1\w2v_s0_w5_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d100_sg1\w2v_s0_w5_d100_sg1.model
Model saved: w2v_s0_w5_d100_sg1


In [20]:
w2v_train_eval(0,5,300,0,analogies)

Model training: w2v_s0_w5_d300_sg0
Model trained: w2v_s0_w5_d300_sg0
Fasttext training time: 1226.0461087226868
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d300_sg0\w2v_s0_w5_d300_sg0.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d300_sg0\w2v_s0_w5_d300_sg0.model
Model saved: w2v_s0_w5_d300_sg0


In [ ]:
#w2v_train_eval(0,5,300,1,analogies)

In [12]:
w2v_train_eval(0,10,100,1,analogies)

Model training: w2v_s0_w10_d100_sg1
Model trained: w2v_s0_w10_d100_sg1
Fasttext training time: 1375.4759013652802
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w10_d100_sg1\w2v_s0_w10_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w10_d100_sg1\w2v_s0_w10_d100_sg1.model
Model saved: w2v_s0_w10_d100_sg1


NameError: name 'n' is not defined

In [18]:
w2v_load_eval(0,10,100,1,analogies)

Loading model from D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w10_d100_sg1\w2v_s0_w10_d100_sg1.model
Model loaded: w2v_s0_w10_d100_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.38095238095238093
evalScoreT3: 0.4523809523809524
mean_sim:  0.7921801585900156
mean_top:  1.263157894736842
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.38095238095238093
evalScoreT5: 0.47619047619047616
mean_sim:  0.7875229209661484
mean_top:  1.4
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.38095238095238093
evalScoreT10: 0.5714285714285714
mean_sim:  0.7708047280708948
mean_top:  2.3333333333333335
evaluating done.


In [19]:
w2v_train_eval(0,15,100,1,analogies)

Model training: w2v_s0_w15_d100_sg1
Model trained: w2v_s0_w15_d100_sg1
Fasttext training time: 1727.2781298160553
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w15_d100_sg1\w2v_s0_w15_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w15_d100_sg1\w2v_s0_w15_d100_sg1.model
Model saved: w2v_s0_w15_d100_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT3: 0.4523809523809524
mean_sim:  0.8070851878116005
mean_top:  1.1578947368421053
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT5: 0.5
mean_sim:  0.7963615145002093
mean_top:  1.4761904761904763
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT10: 0.5714285714285714
mean_sim:  0.787410298983256
mean_top:  2.2083333333333335

In [20]:
w2v_train_eval(0,20,100,1,analogies)

Model training: w2v_s0_w20_d100_sg1
Model trained: w2v_s0_w20_d100_sg1
Fasttext training time: 1984.7928166389465
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w20_d100_sg1\w2v_s0_w20_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w20_d100_sg1\w2v_s0_w20_d100_sg1.model
Model saved: w2v_s0_w20_d100_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.40476190476190477
evalScoreT3: 0.4523809523809524
mean_sim:  0.793669509260278
mean_top:  1.105263157894737
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.40476190476190477
evalScoreT5: 0.47619047619047616
mean_sim:  0.794109097123146
mean_top:  1.25
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.40476190476190477
evalScoreT10: 0.5238095238095238
mean_sim:  0.7853280793536793
mean_top:  1.8181818181818181

In [21]:
w2v_train_eval(0,10,300,1,analogies)

Model training: w2v_s0_w10_d300_sg1
Model trained: w2v_s0_w10_d300_sg1
Fasttext training time: 2053.113043785095
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w10_d300_sg1\w2v_s0_w10_d300_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w10_d300_sg1\w2v_s0_w10_d300_sg1.model
Model saved: w2v_s0_w10_d300_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.40476190476190477
evalScoreT3: 0.4523809523809524
mean_sim:  0.6572419687321311
mean_top:  1.1578947368421053
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.40476190476190477
evalScoreT5: 0.5238095238095238
mean_sim:  0.6435714336958799
mean_top:  1.6363636363636365
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.40476190476190477
evalScoreT10: 0.5714285714285714
mean_sim:  0.6346511021256447
mean_top:  2.2

In [23]:
w2v_train_eval(0,25,100,1,analogies)

Model training: w2v_s0_w25_d100_sg1
Model trained: w2v_s0_w25_d100_sg1
Fasttext training time: 2786.4249250888824
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w25_d100_sg1\w2v_s0_w25_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w25_d100_sg1\w2v_s0_w25_d100_sg1.model
Model saved: w2v_s0_w25_d100_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.4523809523809524
evalScoreT3: 0.4523809523809524
mean_sim:  0.7898313559983906
mean_top:  1.0526315789473684
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.4523809523809524
evalScoreT5: 0.47619047619047616
mean_sim:  0.7903501570224762
mean_top:  1.25
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.4523809523809524
evalScoreT10: 0.47619047619047616
mean_sim:  0.7903501570224762
mean_top:  1.25
evaluating d

In [25]:
w2v_train_eval(0,30,100,1,analogies)

Model training: w2v_s0_w30_d100_sg1
Model trained: w2v_s0_w30_d100_sg1
Fasttext training time: 3158.821650505066
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w30_d100_sg1\w2v_s0_w30_d100_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w30_d100_sg1\w2v_s0_w30_d100_sg1.model
Model saved: w2v_s0_w30_d100_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.38095238095238093
evalScoreT3: 0.42857142857142855
mean_sim:  0.7875840697023604
mean_top:  1.2222222222222223
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.38095238095238093
evalScoreT5: 0.4523809523809524
mean_sim:  0.7817418355690805
mean_top:  1.368421052631579
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.38095238095238093
evalScoreT10: 0.5238095238095238
mean_sim:  0.7726939781145616
mean_top:  2.2

In [26]:
w2v_train_eval(0,15,200,1,analogies)

Model training: w2v_s0_w15_d200_sg1
Model trained: w2v_s0_w15_d200_sg1
Fasttext training time: 2951.1605718135834
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w15_d200_sg1\w2v_s0_w15_d200_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w15_d200_sg1\w2v_s0_w15_d200_sg1.model
Model saved: w2v_s0_w15_d200_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT3: 0.40476190476190477
mean_sim:  0.7219123805270475
mean_top:  1.0
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT5: 0.47619047619047616
mean_sim:  0.7073572248220443
mean_top:  1.5
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT10: 0.5238095238095238
mean_sim:  0.7037821818481792
mean_top:  2.1363636363636362
evaluating 

In [27]:
w2v_train_eval(0,15,300,1,analogies)

Model training: w2v_s0_w15_d300_sg1
Model trained: w2v_s0_w15_d300_sg1
Fasttext training time: 3430.785088777542
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w15_d300_sg1\w2v_s0_w15_d300_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w15_d300_sg1\w2v_s0_w15_d300_sg1.model
Model saved: w2v_s0_w15_d300_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT3: 0.4523809523809524
mean_sim:  0.6743235242994208
mean_top:  1.1578947368421053
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT5: 0.4523809523809524
mean_sim:  0.6743235242994208
mean_top:  1.1578947368421053
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.42857142857142855
evalScoreT10: 0.47619047619047616
mean_sim:  0.6690103381872177
mean_top:  1.

In [28]:
w2v_train_eval(0,15,50,1,analogies)

Model training: w2v_s0_w15_d50_sg1
Model trained: w2v_s0_w15_d50_sg1
Fasttext training time: 2167.2349922657013
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w15_d50_sg1\w2v_s0_w15_d50_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w15_d50_sg1\w2v_s0_w15_d50_sg1.model
Model saved: w2v_s0_w15_d50_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.3333333333333333
evalScoreT3: 0.40476190476190477
mean_sim:  0.8560775728786693
mean_top:  1.1764705882352942
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.3333333333333333
evalScoreT5: 0.47619047619047616
mean_sim:  0.8460548669099808
mean_top:  1.7
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.3333333333333333
evalScoreT10: 0.5238095238095238
mean_sim:  0.8376778933134946
mean_top:  2.4545454545454546
evalua

In [12]:
w2v_train_eval(0,5,50,1,analogies)

Model training: w2v_s0_w5_d50_sg1
Model trained: w2v_s0_w5_d50_sg1
Fasttext training time: 944.9098114967346
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d50_sg1\w2v_s0_w5_d50_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d50_sg1\w2v_s0_w5_d50_sg1.model
Model saved: w2v_s0_w5_d50_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2619047619047619
evalScoreT3: 0.35714285714285715
mean_sim:  0.8724554975827535
mean_top:  1.4
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2619047619047619
evalScoreT5: 0.4523809523809524
mean_sim:  0.854242553836421
mean_top:  2.1052631578947367
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2619047619047619
evalScoreT10: 0.47619047619047616
mean_sim:  0.8518807202577591
mean_top:  2.45
evaluating done.


In [13]:
w2v_train_eval(0,5,200,1,analogies)

Model training: w2v_s0_w5_d200_sg1
Model trained: w2v_s0_w5_d200_sg1
Fasttext training time: 1148.4212937355042
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d200_sg1\w2v_s0_w5_d200_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d200_sg1\w2v_s0_w5_d200_sg1.model
Model saved: w2v_s0_w5_d200_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.30952380952380953
evalScoreT3: 0.42857142857142855
mean_sim:  0.6983287533124288
mean_top:  1.4444444444444444
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.30952380952380953
evalScoreT5: 0.5238095238095238
mean_sim:  0.6858843104405836
mean_top:  1.9545454545454546
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.30952380952380953
evalScoreT10: 0.5714285714285714
mean_sim:  0.6775408635536829
mean_top:  2.416666

In [14]:
w2v_train_eval(0,5,400,1,analogies)

Model training: w2v_s0_w5_d400_sg1
Model trained: w2v_s0_w5_d400_sg1
Fasttext training time: 2140.53084897995
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d400_sg1\w2v_s0_w5_d400_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d400_sg1\w2v_s0_w5_d400_sg1.model
Model saved: w2v_s0_w5_d400_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2619047619047619
evalScoreT3: 0.35714285714285715
mean_sim:  0.6152929107348124
mean_top:  1.3333333333333333
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2619047619047619
evalScoreT5: 0.42857142857142855
mean_sim:  0.6034082637892829
mean_top:  1.8888888888888888
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2619047619047619
evalScoreT10: 0.5
mean_sim:  0.5961940515609014
mean_top:  2.619047619047619
evaluatin

In [15]:
w2v_train_eval(0,5,500,1,analogies)

Model training: w2v_s0_w5_d500_sg1
Model trained: w2v_s0_w5_d500_sg1
Fasttext training time: 3052.3342111110687
D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d500_sg1\w2v_s0_w5_d500_sg1.model not existed, creating...
Saving model to D:\german_legal_WE\WEs\gensim_w2v\w2v_s0_w5_d500_sg1\w2v_s0_w5_d500_sg1.model
Model saved: w2v_s0_w5_d500_sg1
evaluating T3
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2857142857142857
evalScoreT3: 0.40476190476190477
mean_sim:  0.5845430633601021
mean_top:  1.5294117647058822
evaluating done.
evaluating T5
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2857142857142857
evalScoreT5: 0.4523809523809524
mean_sim:  0.5750521076352972
mean_top:  1.8421052631578947
evaluating done.
evaluating T10
d4u:False
Quadruplets with out-of-vocabulary words:  60.747663551401864
evalScoreT1: 0.2857142857142857
evalScoreT10: 0.5
mean_sim:  0.5704353111130851
mean_top:  2.2857142857142856
evaluat

## check vocabulary size and thevector diemsion

In [14]:
modelName,model=load_model(0,5,100,1)

Loading model from C:\Users\KlaraRuanQian\Desktop\BA\german_legal_WE\WEs\gensim_fasttext\ft_s0_w5_d100_sg1\ft_s0_w5_d100_sg1.model
Model loaded: ft_s0_w5_d100_sg1


In [15]:
print("Model: "+modelName)
print("Vocabulary size", len(list(model.wv.vocab.keys())))
print("Word vector length:", len(model.wv["Mann"]))

Model: ft_s0_w5_d100_sg1
Vocabulary size 695675
Word vector length: 100


In [16]:
modelName,model=load_model(1,5,100,1)

Loading model from C:\Users\KlaraRuanQian\Desktop\BA\german_legal_WE\WEs\gensim_fasttext\ft_s1_w5_d100_sg1\ft_s1_w5_d100_sg1.model
Model loaded: ft_s1_w5_d100_sg1


In [17]:
print("Model: "+modelName)
print("Vocabulary size", len(list(model.wv.vocab.keys())))
print("Word vector length:", len(model.wv["Mann"]))

Model: ft_s1_w5_d100_sg1
Vocabulary size 282731
Word vector length: 100


In [18]:
type(model)

gensim.models.fasttext.FastText

In [19]:
type(model.wv)

gensim.models.keyedvectors.FastTextKeyedVectors

In [1]:
def check_vsd(modelName,model):
    print("Vocabulary size", len(list(model.wv.vocab.keys())))
    print("Word vector length:", len(model.wv["Mann"]))
    

## continue training prtrained models on legal corpus:

In [ ]:
pt_ft_d300_ky = FastText.load_fasttext_format(modelPath4,full_model=True)